In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Conv2D,
    MaxPooling2D,
    GlobalAveragePooling2D,
    Flatten,
    Dropout,
    BatchNormalization,
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.callbacks import ModelCheckpoint, EarlyStopping
from glob import glob

In [ ]:
#dataset of fake real recognition
train_path = './tranning'
validation_path = './testing'

In [ ]:
# no. of classes in recognition data
className = glob(train_path + "/*")
NumberofClass = len(className)
print("NumberofClass:", NumberofClass)

In [ ]:
# data augumentation 
train_datagen = ImageDataGenerator(rescale=1/255)    
validation_datagen=ImageDataGenerator(rescale=1/255)
test_datagen=ImageDataGenerator(rescale=1/255)

In [ ]:
batch_size = 10
# recognition data

train_datagen = ImageDataGenerator(rescale=1/255,
                  shear_range=0.3,
                  horizontal_flip=True,
                  zoom_range=0.3
                  )
val_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
                train_path,
                target_size=(224,224),
                batch_size=batch_size,
                color_mode="rgb",
                class_mode="categorical"
                )

val_generator = val_datagen.flow_from_directory(
                validation_path,
                target_size=(224,224),
                batch_size=batch_size,
                color_mode="rgb",
                class_mode="categorical"
                )

In [ ]:
# Get a batch of images and labels from the training generator
images, labels = next(train_generator)

# Display the original and augmented images for the first 5 images
plt.figure(figsize=(15, 10))
for i in range(5):
    # Original Image
    plt.subplot(2, 5, i + 1)
    plt.imshow(images[i])
    plt.title(f'Augmented')
    plt.axis('off')

    # Augmented Image
    augmented_image = train_datagen.random_transform(images[i])
    augmented_image = np.clip(augmented_image, 0, 1)  # Clip values to [0, 1] range
    
    plt.subplot(2, 5, i + 6)
    plt.imshow(augmented_image)
    plt.title(f'Original')
    plt.axis('off')

plt.show()

In [ ]:
train_generator.class_indices

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(NumberofClass, activation='softmax'))

# Use an adaptive learning rate such as Adam
opt = Adam(learning_rate=0.001)

model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

# Implement data augmentation and create generator objects
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Adjust the batch_size and target_size based on your dataset
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

model.fit(
    train_generator,
    steps_per_epoch=len(train_generator) // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator) // validation_generator.batch_size,
    class_weight={0: 1, 1: 1.5}  # Adjust the class weights based on your dataset
)

In [ ]:
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_generator,
                    validation_data = val_generator,
                    batch_size = 108,
                    epochs = 20)

In [ ]:
# Plot for fake vs real classification
plt.figure(figsize=(12, 6))

plt.plot(history.history['accuracy'], 'go--', label='Training Accuracy')
plt.plot(history.history['val_accuracy'], 'ro--', label='Validation Accuracy')

plt.title('Training and Validation Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Plot for loss vs val_loss
plt.figure(figsize=(12, 6))

plt.plot(history.history['loss'], 'go--', label='Training Loss')
plt.plot(history.history['val_loss'], 'ro--', label='Validation Loss')

plt.title('Training and Validation Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
val_generator.reset()

# Use model.evaluate to get predictions
results = model.evaluate(val_generator, steps=len(val_generator), verbose=0)

# 'results' will contain loss and accuracy
loss, accuracy = results

print(f'Validation Accuracy: {accuracy * 100:.2f}%')


In [ ]:
# Load the custom image
custom_img_path = 'C:/Users/12499/Pictures/Screenshots/Screenshot 2023-11-27 225217.png'  # Replace with your image path
custom_img = load_img(custom_img_path, target_size=(224, 224))

# Convert the image to a NumPy array and normalize
custom_img_array = img_to_array(custom_img) / 255.0
custom_img_array = np.expand_dims(custom_img_array, axis=0)  # Add an extra dimension for batch size

In [ ]:
# Make predictions
predictions = model.predict(custom_img_array)

# Get the class with the highest probability
predicted_class = np.argmax(predictions)

# Map the class index to the corresponding class label (real or fake)
class_labels = {0: 'Real', 1: 'Fake'}
predicted_label = class_labels[predicted_class]

print(f'The predicted class is: {predicted_label}')

In [ ]:
# Load the custom image
custom_img_path = 'C:/Users/12499/Pictures/Screenshots/Screenshot 2023-11-27 183138.png'# Replace with your image path
custom_img = load_img(custom_img_path, target_size=(224, 224))

# Convert the image to a NumPy array and normalize
custom_img_array = img_to_array(custom_img) / 255.0
custom_img_array = np.expand_dims(custom_img_array, axis=0)  # Add an extra dimension for batch size
# Make predictions
predictions = model.predict(custom_img_array)

# Get the class with the highest probability
predicted_class = np.argmax(predictions)

# Map the class index to the corresponding class label (real or fake)
class_labels = {0: 'Real', 1: 'Fake'}
predicted_label = class_labels[predicted_class]

print(f'The predicted class is: {predicted_label}')